In [1]:
import pandas as pd
from IPython.display import display, Markdown

## Data

In [2]:
nlexique = pd.read_csv('../data/nlexique.csv')
nlexique = nlexique[['lexeme', 'sg', 'gen']].rename(columns={'sg': 'phon'}).dropna().reset_index(drop=True)
nlexique

,lexeme,phon,gen
0,à-côté,akOte,m
1,à-coup,aku,m
2,à-peu-près,apØpʁɛ,m
3,à-pic,apik,m
4,à-plat,apla,m
...,...,...,...
30999,zurichois,zyʁikwa,m
31000,zydeco,zidəko,f
31001,zygoma,zigOma,m
31002,zygote,zigɔt,m


In [3]:
echantinom = pd.read_csv('../data/Echantinom-full-20210902.csv')
echantinom = echantinom[~echantinom['lemma'].isin(nlexique[nlexique['gen'] == 'b']['lexeme'])] #[['lemma', 'phon', 'gen']]
echantinom

,lemma,gen,phon,freq_lex_books,freq_lex_subtitles,freq_frcow,last_process_broad,last_process_narrow,prefix,compound,...,autonomous_base,base_stem_phon,sfx_allomorph,der_stem_phon,edit_distance,pattern,pattern_tf,pattern_rel_tf,base_der_sim,offset_sim
0,berlingue,m,bɛʁ.lɛ̃g,0.34,0.00,34,nonconcat,apocope,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,corton,m,kɔʁ.tɔ̃,0.27,0.03,398,suffix,suffix,0,0,...,True,kuʁ,ɔ̃,kɔʁt,2,_u_~_ɔ_tɔ̃,1,0.015625,0.222162783145905,0.158108526129264
2,dabuche,f,da.byʃ,0.54,0.00,3,suffix,suffix,0,0,...,True,UNKNOWN,yʃ,dab,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN
3,faf,m,faf,0.88,0.07,3422,nonconcat,apocope,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,gail,f,gaj,0.61,0.00,2471,simplex,native_simplex,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,palu,m,pa.ly,0.14,0.80,1431,nonconcat,apocope,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4996,talc,m,talk,1.49,1.40,2775,simplex,borrowing,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4997,sauvetage,m,sO.və.taʒ,3.72,8.32,60875,suffix,suffix,0,0,...,True,sOv,aʒ,sOvət,2,_~_ətaʒ,1,0.011494252873563,0.252674728631973,0.614848479997584
4998,plaid,m,plɛd,1.15,0.34,2889,simplex,borrowing,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
orth_preds_x10 = pd.read_csv('../results/echantinom_orth_preds_x10.csv')
orth_preds_x10.rename(columns={"Predicted Gender": "orth_pred", "True Gender": "true", "Form": 'lemma'}, inplace=True)
orth_preds_x10

,lemma,orth_pred,true,Class Probabilities,Set,Run
0,an,m,m,"[('n', {'f': 0.4085073173046112, 'm': 0.591492...",test,1
1,bi,m,m,"[('i', {'f': 0.16074861586093903, 'm': 0.83925...",test,1
2,as,m,m,"[('s', {'f': 0.2868475317955017, 'm': 0.713152...",test,1
3,té,m,m,"[('é', {'f': 0.3696044683456421, 'm': 0.630395...",test,1
4,pie,f,f,"[('e', {'f': 0.11107771843671799, 'm': 0.88892...",test,1
...,...,...,...,...,...,...
45315,approvisionnement,m,m,"[('t', {'f': 0.07809196412563324, 'm': 0.92190...",test,10
45316,sous-alimentation,f,f,"[('n', {'f': 0.38623788952827454, 'm': 0.61376...",test,10
45317,cul-de-basse-fosse,m,m,"[('e', {'f': 0.23680555820465088, 'm': 0.76319...",test,10
45318,marie-couche-toi-là,f,f,"[('à', {'f': 0.28149473667144775, 'm': 0.71850...",test,10


In [5]:
phon_preds_x10 = pd.read_csv('../results/echantinom_phon_preds_x10.csv')
phon_preds_x10.rename(columns={"Predicted Gender": "phon_pred", "True Gender": "true", "Form": 'lemma'}, inplace=True)
phon_preds_x10

,lemma,phon_pred,true,Class Probabilities,Set,Run
0,ɛ,m,f,"[('ɛ', {'f': 0.14882417023181915, 'm': 0.85117...",test,1
1,u,f,m,"[('u', {'f': 0.3157658278942108, 'm': 0.684234...",test,1
2,ɛ,m,m,"[('ɛ', {'f': 0.14882417023181915, 'm': 0.85117...",test,1
3,o,m,f,"[('o', {'f': 0.5670557618141174, 'm': 0.432944...",test,1
4,ba,m,m,"[('a', {'f': 0.39993923902511597, 'm': 0.60006...",test,1
...,...,...,...,...,...,...
45315,kɔ̃fidɑ̃sjalite,f,f,"[('e', {'f': 0.5326544642448425, 'm': 0.467345...",test,10
45316,ɛ̃kɔ̃patibilite,f,f,"[('e', {'f': 0.5326544642448425, 'm': 0.467345...",test,10
45317,suzalimɑ̃tasjɔ̃,f,f,"[('̃', {'f': 0.6368642449378967, 'm': 0.363135...",test,10
45318,tʁɑ̃splɑ̃tasjɔ̃,f,f,"[('̃', {'f': 0.6368642449378967, 'm': 0.363135...",test,10


## Orth/phon agreement vs. disagreement

In [6]:
orth_incorrect = orth_preds_x10[orth_preds_x10.orth_pred != orth_preds_x10.true]
orth_incorrect

,lemma,orth_pred,true,Class Probabilities,Set,Run
42,foi,m,f,"[('i', {'f': 0.16074861586093903, 'm': 0.83925...",test,1
43,ire,m,f,"[('e', {'f': 0.11107771843671799, 'm': 0.88892...",test,1
44,nef,m,f,"[('f', {'f': 0.43117114901542664, 'm': 0.56882...",test,1
48,ive,m,f,"[('e', {'f': 0.11107771843671799, 'm': 0.88892...",test,1
58,glu,m,f,"[('u', {'f': 0.37645187973976135, 'm': 0.62354...",test,1
...,...,...,...,...,...,...
45202,franc-maçonne,m,f,"[('e', {'f': 0.23680555820465088, 'm': 0.76319...",test,10
45241,monte-en-l'air,f,m,"[('r', {'f': 0.29656296968460083, 'm': 0.70343...",test,10
45269,contre-attaque,m,f,"[('e', {'f': 0.23680555820465088, 'm': 0.76319...",test,10
45288,transatlantique,f,m,"[('e', {'f': 0.23680555820465088, 'm': 0.76319...",test,10


In [7]:
orth_correct = orth_preds_x10[orth_preds_x10.orth_pred == orth_preds_x10.true]
orth_correct['phon_pred'] = phon_preds_x10.phon_pred 
pd.crosstab(orth_correct.orth_pred, orth_correct.phon_pred)

C:\Users\ninan\AppData\Local\Temp\ipykernel_5344\2831827995.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orth_correct['phon_pred'] = phon_preds_x10.phon_pred


phon_pred,f,m
orth_pred,,
f,5328,9151
m,9395,17216


### Phon/orth agreement vs. disagreement

In [8]:
phon_incorrect = phon_preds_x10[phon_preds_x10.phon_pred != phon_preds_x10['true']]
phon_incorrect

,lemma,phon_pred,true,Class Probabilities,Set,Run
0,ɛ,m,f,"[('ɛ', {'f': 0.14882417023181915, 'm': 0.85117...",test,1
1,u,f,m,"[('u', {'f': 0.3157658278942108, 'm': 0.684234...",test,1
3,o,m,f,"[('o', {'f': 0.5670557618141174, 'm': 0.432944...",test,1
5,ʁu,m,f,"[('u', {'f': 0.3157658278942108, 'm': 0.684234...",test,1
7,œj,f,m,"[('j', {'f': 0.06026039645075798, 'm': 0.93973...",test,1
...,...,...,...,...,...,...
45277,tʁɑ̃zatlɑ̃tik,f,m,"[('k', {'f': 0.41414210200309753, 'm': 0.58585...",test,10
45284,kɔ̃tʁəlamɔ̃tʁ,f,m,"[('ʁ', {'f': 0.28374531865119934, 'm': 0.71625...",test,10
45286,pɛʁpɑ̃dikylɛʁ,m,f,"[('ʁ', {'f': 0.28374531865119934, 'm': 0.71625...",test,10
45297,mOdysOpEʁɑ̃di,f,m,"[('i', {'f': 0.3824596405029297, 'm': 0.617540...",test,10


In [9]:
phon_correct = phon_preds_x10[phon_preds_x10.phon_pred == phon_preds_x10.true]
phon_correct['orth_pred'] = orth_preds_x10.orth_pred 
pd.crosstab(phon_correct.phon_pred, phon_correct.orth_pred)


C:\Users\ninan\AppData\Local\Temp\ipykernel_5344\2820111615.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phon_correct['orth_pred'] = orth_preds_x10.orth_pred


orth_pred,f,m
phon_pred,,
f,4487,7383
m,9014,15440


### Orth/phon agreement vs. disagreement

In [10]:
orth_incorrect = orth_preds_x10[orth_preds_x10.orth_pred != orth_preds_x10['true']]
orth_incorrect

,lemma,orth_pred,true,Class Probabilities,Set,Run
42,foi,m,f,"[('i', {'f': 0.16074861586093903, 'm': 0.83925...",test,1
43,ire,m,f,"[('e', {'f': 0.11107771843671799, 'm': 0.88892...",test,1
44,nef,m,f,"[('f', {'f': 0.43117114901542664, 'm': 0.56882...",test,1
48,ive,m,f,"[('e', {'f': 0.11107771843671799, 'm': 0.88892...",test,1
58,glu,m,f,"[('u', {'f': 0.37645187973976135, 'm': 0.62354...",test,1
...,...,...,...,...,...,...
45202,franc-maçonne,m,f,"[('e', {'f': 0.23680555820465088, 'm': 0.76319...",test,10
45241,monte-en-l'air,f,m,"[('r', {'f': 0.29656296968460083, 'm': 0.70343...",test,10
45269,contre-attaque,m,f,"[('e', {'f': 0.23680555820465088, 'm': 0.76319...",test,10
45288,transatlantique,f,m,"[('e', {'f': 0.23680555820465088, 'm': 0.76319...",test,10


In [11]:
orth_correct = orth_preds_x10[orth_preds_x10.orth_pred == orth_preds_x10.true]
orth_correct['phon_pred'] = phon_preds_x10.phon_pred 
pd.crosstab(orth_correct.orth_pred, orth_correct.phon_pred)


C:\Users\ninan\AppData\Local\Temp\ipykernel_5344\2930785969.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orth_correct['phon_pred'] = phon_preds_x10.phon_pred


phon_pred,f,m
orth_pred,,
f,5328,9151
m,9395,17216
